In [ ]:
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, label_binarize
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from bayes_opt import BayesianOptimization
import warnings
import time
import datetime

In [ ]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=9999982357774af00304707ff63c916c75f3baed4ec3e43ca8385bd7f5459ead
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [ ]:
def XGB_CV(max_depth,
           gamma,
           min_child_weight,
           max_delta_step,
           subsample,
           colsample_bytree,
           learning_rate,
         ):
    time2 = time.time()
    global AUCbest
    global ITERbest
    
    folds = 10
    cv_score = 0
    xgb = XGBClassifier(max_depth = int(max_depth),
                        gamma = gamma,
                        learning_rate = learning_rate,
                        subsample = max(min(subsample, 1), 0),
                        colsample_bytree = max(min(colsample_bytree, 1), 0),
                        min_child_weight = min_child_weight,
                        max_delta_step = int(max_delta_step),
                        n_estimators = 20000,
                        random_state=1234, 
                        tree_method='gpu_hist' ,
                        silent=True)
    xgb.fit(X_train, y_train,
            early_stopping_rounds = 100,
            eval_set=[(X_test, y_test)], 
            eval_metric=custom_eval, verbose=False)
    
    
    val_score = -xgb.evals_result()['validation_0']['roc_auc'][-1]
    print(' Stopped after %d iterations with val-auc = %f val-gini = %f' % ( len(xgb.evals_result()['validation_0']['roc_auc']), val_score, (val_score*2-1)) )
    if ( val_score > AUCbest ):
        AUCbest = val_score
        ITERbest = len(xgb.evals_result()['validation_0']['roc_auc'])
    time2 -=time.time()

    return (val_score*2) - 1

In [ ]:
def multiclass_roc_auc_score(y_test, y_pred_proba, average=None): #average='macro' when you wanna get mean
  lb = LabelBinarizer()
  lb.fit(y_test)
  y_test = lb.transform(y_test)
  #y_pred = lb.transform(y_pred)   #if y_pred_proba is not a probability
  roc_auc = roc_auc_score(y_test, y_pred_proba, average=average)


In [ ]:
def custom_eval(pred, dtrain):
  labels = dtrain.get_label()
  lb = LabelBinarizer()
  lb.fit(labels)
  label = lb.transform(labels)
  return 'roc_auc' , -roc_auc_score(label, pred)

In [ ]:
%cd /content/drive/My Drive/samsung_card/preprocess  

path = '../data/'
test_file = 'df_merged.csv'
df = pd.read_csv(path+test_file, index_col='cst_id_di')

/content/drive/My Drive/samsung_card/preprocess


In [ ]:
XGB_BO = BayesianOptimization(XGB_CV, {
                                    'max_depth': (2, 12),
                                    'gamma': (0.001, 10.0),
                                    'min_child_weight': (0, 20),
                                    'max_delta_step': (0, 10),
                                    'subsample': (0.4, 1.0),
                                    'colsample_bytree' :(0.4, 1.0),
                                    'learning_rate' : (0.01, 0.1)
                                    })

In [ ]:
X = df.iloc[:, 1:].values; y = df.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1234)


In [ ]:
AUCbest = -1.
ITERbest = 0
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    XGB_BO.maximize(init_points=2, n_iter=30, acq='ei', xi=0.0)

|   iter    |  target   | colsam... |   gamma   | learni... | max_de... | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------------------------------
 Stopped after 408 iterations with val-auc = 0.886572 val-gini = 0.773144
|  1        |  0.7731   |  0.4939   |  4.715    |  0.06635  |  5.155    |  9.919    |  9.163    |  0.4351   |
 Stopped after 330 iterations with val-auc = 0.886273 val-gini = 0.772546
|  2        |  0.7725   |  0.9182   |  1.465    |  0.09187  |  7.554    |  6.131    |  0.1714   |  0.7001   |
 Stopped after 333 iterations with val-auc = 0.887737 val-gini = 0.775474
|  3        |  0.7755   |  0.4828   |  9.944    |  0.07899  |  6.583    |  10.98    |  15.32    |  0.6641   |
 Stopped after 2180 iterations with val-auc = 0.888194 val-gini = 0.776388
|  4        |  0.7764   |  0.4899   |  9.495    |  0.04922  |  9.682    |  2.199    |  19.92    |  0.8426   |
 Stopped after 5503 iterations with val-auc

In [ ]:
def XGB_CV(max_depth,
           gamma,
           min_child_weight,
           max_delta_step,
           subsample,
           colsample_bytree,
           learning_rate,
         ):

    global AUCbest
    global ITERbest

    folds = 10
    cv_score = 0

    lg = lgb.LGBMClassifier(class_weight=None,
                        colsample_bytree=0.9234, learning_rate=0.01, max_depth=20, 
                        min_child_weight=0.1, min_split_gain=0.0, n_estimators=150, 
                        n_jobs=-1, num_leaves=13, random_state=42, silent=True)
    

    xgb = XGBClassifier(max_depth = int(max_depth),
                        gamma = gamma,
                        learning_rate = learning_rate,
                        subsample = max(min(subsample, 1), 0),
                        colsample_bytree = max(min(colsample_bytree, 1), 0),
                        min_child_weight = min_child_weight,
                        max_delta_step = int(max_delta_step),
                        n_estimators = 20000,
                        random_state=1234, 
                        tree_method='gpu_hist' ,
                        silent=True)
    xgb.fit(X_train, y_train,
            verbose=False,  
            early_stopping_rounds = 100,
            eval_set=[(X_test, y_test)],
            eval_metric=custom_eval, )

    
    val_score = xgb.evals_result()['validation_0']['roc_auc'][-1]
    print(' Stopped after %d iterations with val-auc = %f val-gini = %f' % ( len(xgb.evals_result()['validation_0']['roc_auc']), val_score, (val_score*2-1)) )
    if ( val_score > AUCbest ):
        AUCbest = val_score
        ITERbest = len(xgb.evals_result()['validation_0']['roc_auc'])

    return (val_score*2) - 1